In [0]:
from google.colab import drive
drive.mount('/content/drive/')
!ls

# change working directory
import os
# %cd ../
os.chdir("drive/My Drive/Covid-19")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/
drive  sample_data


In [0]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
import time

# !pip install geopy
# from geopy.geocoders import Nominatim
# geolocator = Nominatim(user_agent="specify_your_app_name_here")

!pip install allennlp==1.0.0rc1 allennlp-models==1.0.0rc1
import allennlp_models
import allennlp_models.sentiment
from allennlp.predictors.predictor import Predictor
predictor = Predictor.from_path(
    "https://storage.googleapis.com/allennlp-public-models/sst-roberta-large-2020.02.17.tar.gz",
    cuda_device=0)

# from countries import *

!pip install pycountry-convert
import pycountry_convert as pc

!pip install langdetect
from langdetect import detect

# !pip install googletrans
# from googletrans import Translator
# translator = Translator()

# !pip install mtranslate
# from mtranslate import translate

import re
from clean_tweets import clean_string

# !pip install pandarallel
# from pandarallel import pandarallel
# pandarallel.initialize(progress_bar=True)

def preprocess_pipeline(row):
    text = str(row.loc['text'])
    # TODO change name to location
    # location = str(row.loc['init-country'][:])

    # Clean text
    text, hashtags = clean_string(text)
    hashtags = ' '.join(hashtags)

    # Translate text
    try:
      lang = detect(text)
    except:
      lang = 'na'
    # text = translator.translate(text, src=lang).text

    # Sentiment of text
    if lang == 'en':
      sentiment_dict = predictor.predict(text)
      sentiment_class = sentiment_dict['label']
      negative_prob = sentiment_dict['probs'][1]
    else:
      sentiment_class = 9
      negative_prob = 0.5


    # Find country of location
    # geolocation = geolocator.geocode(location, language='en')
    # country = geolocation.address.split(',')[-1].strip()
    # country = location

    # Find continent
    # country_code = pc.country_name_to_country_alpha2(country, cn_name_format="default")
    # continent = countries.loc[country_code, 'continent']

    return pd.Series([text, lang, hashtags, sentiment_class, negative_prob],
                     index=['text', 'lang', 'hashtags', 'sentiment', 'negative sentiment probability'])


In [0]:
%%time
# code to list all files and show total number of collected tweets
from os import listdir
from os.path import isfile, join

mypath = 'tweets'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

savepath = 'covid-final-large-dataset-30-06-04_correct.csv.gz'
for filepath in onlyfiles:
    df = pd.read_csv(mypath + '/' + filepath)

    df = df.merge(df.progress_apply(preprocess_pipeline, axis=1), left_index=True, right_index=True)
    df.drop(['text_x'], axis=1, inplace=True)
    df = df[df['lang'] == 'en']
    df.sort_values(by=['date','time'], inplace=True)
    df.to_csv(savepath, index=False, compression='gzip', mode='a')



100%|██████████| 93024/93024 [58:01<00:00, 26.72it/s]


KeyError: ignored

In [0]:
df.head()